In [1]:
import requests
import time
import random
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from IPython.core.display import display, HTML
import openpyn
import numpy as np
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

C:\Users\Aldie\AppData\Local\Temp\ipykernel_256756\1185670893.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [60]:
VS_Restaurant = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])

In [3]:
def extractor(url, category, maindf):
    #Consider first the first page of the website
    main_url = url

    #Preventing IP block again
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    #Open the website using BS4 and get the main page only
    MAIN_response = requests.get(url, headers = headers)
    MAIN_soup = BeautifulSoup(MAIN_response.content, 'html.parser')
    
    #Extract all URLs of businesses under the first page
    LIST_biz = MAIN_soup.find_all('div', 'company_header')
    pageX_biz = []
    for entry in LIST_biz:
        url = f'https://www.businesslist.ph' + entry.find('a')['href']
        pageX_biz.append(url)
        
    #Manually open them
    for URL_biz in pageX_biz:
        URL_response = requests.get(URL_biz, headers = headers)
        soup = BeautifulSoup(URL_response.content, 'html.parser')

        #Temporary container
        list = []
        
        #Get all relevant information
        CATEGORY = category
        BIZNAME = (soup.find('div', {'id': 'company_name'}).get_text(strip=True) if soup.find('div', {'id': 'company_name'}) else 'N/A')
        LOCATION = (soup.find('div', {'id': 'company_address'}).get_text(strip=True) if soup.find('div', {'id': 'company_address'}) else 'N/A')
        PHONE = (soup.find_all('div', 'text')[2].text if soup.find_all('div', 'text') else 'N/A')
        WEBSITE = (f'https://' + soup.find('div', 'text weblinks').text if soup.find('div', 'text weblinks') else 'N/A')
        RATING = (soup.find_all('span', {'itemprop': 'ratingValue'})[0].text if soup.find_all('span', {'itemprop': 'ratingValue'}) else 'N/A')
        ALL_REVIEWS = [review.text for review in soup.find_all('div', {'itemprop': 'reviewBody'})] or ['N/A']
        MAINLINK = URL_biz
        
        #Append all entries
        list.append(CATEGORY)
        list.append(BIZNAME)
        list.append(LOCATION)
        list.append(PHONE)
        list.append(WEBSITE)
        list.append(RATING)
        list.append(ALL_REVIEWS)
        list.append(MAINLINK)

        #Append them
        list = [list]
        TEMP_df = pd.DataFrame(list, columns = maindf.columns)
        maindf = pd.concat([TEMP_df, maindf], ignore_index=True)

        #Print test
        #print(list)
        print(f'Done with {URL_biz}')
        
        #Interval to PREVENT IP ADDRESS BLOCKING
        time.sleep(random.randint(12, 15))

    #Print test to know our current page
    print(f'Done with {main_url}')

    #Interval to PREVENT IP ADDRESS BLOCKING
    time.sleep(random.randint(12, 15))

    #Go to the next page by doing recursion
    if MAIN_soup.find('div', class_='scroller scroller_with_ul').find('a', rel='next'):
        new_url = f'https://www.businesslist.ph' + MAIN_soup.find('div', class_='scroller scroller_with_ul').find('a', rel='next')['href']
        maindf = extractor(new_url, category, maindf)
        
    return maindf

In [54]:
VS_Restaurant = extractor('https://www.businesslist.ph/category/vehicle-services', 'Vehicle Service', VS_Restaurant)

Done with https://www.businesslist.ph/company/301164/chevrolet-bf-paranaque
Done with https://www.businesslist.ph/company/300373/lipat-bahay-trucking-services
Done with https://www.businesslist.ph/company/301248/motor-atbp-sa-lipa
Done with https://www.businesslist.ph/company/297673/the-movers-777lipat-bahay-
Done with https://www.businesslist.ph/company/300844/amv-rent-a-car
Done with https://www.businesslist.ph/company/297508/mts-car-rental
Done with https://www.businesslist.ph/company/300623/star-motor-armoring-technologies-inc
Done with https://www.businesslist.ph/company/300374/a-star-cargo-lipat-bahay-trucking-services-moving
Done with https://www.businesslist.ph/company/301547/europa-yachts-philippines-inc
Done with https://www.businesslist.ph/company/296541/jessica-v-ent-lipat-bahay-and-trucking-company
Done with https://www.businesslist.ph/company/145856/motortech-inc
Done with https://www.businesslist.ph/company/299484/brand-new-and-used-cars-for-sale-in-philippinestsikot
Don

In [55]:
VS_Restaurant.to_csv('VS_Restaurant.csv')

In [68]:
Construction_Services = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])
Construction_Services = extractor('https://www.businesslist.ph/category/construction-services', 'Construction Services', Construction_Services)
Construction_Services.to_csv('Construction_Services.csv')

Done with https://www.businesslist.ph/company/214862/tdt-powersteel-corporation
Done with https://www.businesslist.ph/company/298406/casparsteel-roofing-inc
Done with https://www.businesslist.ph/company/300025/construction-cebu-company
Done with https://www.businesslist.ph/company/125941/allied-inspection-corrosion-services-alincor
Done with https://www.businesslist.ph/company/264672/aaaa-synergy-jama-konstrukt-inc
Done with https://www.businesslist.ph/company/302133/alphaconstruct-corporation
Done with https://www.businesslist.ph/company/299931/castlepower-solutions-philippines-incorporated
Done with https://www.businesslist.ph/company/263988/addplus-solutions-group-inc
Done with https://www.businesslist.ph/company/296492/gruppo-struktura-inc
Done with https://www.businesslist.ph/company/297922/hqc-building-equipments-corp
Done with https://www.businesslist.ph/company/302475/1st-top-notch-construction-corp
Done with https://www.businesslist.ph/company/298425/lazaro-and-son-constructio

In [71]:
General_Business = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])
General_Business = extractor('https://www.businesslist.ph/category/general-business', 'General_Business', General_Business)
General_Business.to_csv('General_Business.csv')

Done with https://www.businesslist.ph/company/305245/peak-outsourcing-philippines
Done with https://www.businesslist.ph/company/301077/manila-gps-trackers
Done with https://www.businesslist.ph/company/303235/princess-cut-jewelry-buyer
Done with https://www.businesslist.ph/company/302349/boomering-inc
Done with https://www.businesslist.ph/company/302967/ad-plumbing-and-electrical-services
Done with https://www.businesslist.ph/company/106415/edi-staffbuilders-international
Done with https://www.businesslist.ph/company/281183/camox-philippines-incorporated
Done with https://www.businesslist.ph/company/264672/aaaa-synergy-jama-konstrukt-inc
Done with https://www.businesslist.ph/company/299322/malabanan-siphoning-pozo-negro
Done with https://www.businesslist.ph/company/298837/loan-consultant
Done with https://www.businesslist.ph/company/298278/kp-convertech-inc
Done with https://www.businesslist.ph/company/299217/ecfm-accounting-and-auditing-services
Done with https://www.businesslist.ph/co

In [5]:
Employment_Services = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])
Employment_Services = extractor('https://www.businesslist.ph/category/employment-agencies', 'Employment Services', Employment_Services)
Employment_Services.to_csv('Employment_Services.csv')

Done with https://www.businesslist.ph/company/106474/abba-personnel-services-inc
Done with https://www.businesslist.ph/company/304368/topserve-service-solutions-inc
Done with https://www.businesslist.ph/company/106415/edi-staffbuilders-international
Done with https://www.businesslist.ph/company/304376/e-gmp-international-corporation
Done with https://www.businesslist.ph/company/304369/ikon-solutions-asia-inc
Done with https://www.businesslist.ph/company/106462/industrial-personnel-and-management-services-inc
Done with https://www.businesslist.ph/company/181465/monroe-consulting-group
Done with https://www.businesslist.ph/company/203588/jedegal-intl-manpower-services-incprivate-employment-agency
Done with https://www.businesslist.ph/company/303032/jobaxy-brand-yourself
Done with https://www.businesslist.ph/company/304375/9k-international-manpower-and-recruitment-services-corp
Done with https://www.businesslist.ph/company/281183/camox-philippines-incorporated
Done with https://www.busine

In [7]:
Advertising = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])
Advertising = extractor('https://www.businesslist.ph/category/advertising', 'Advertising', Advertising)
Advertising.to_csv('Advertising.csv')

Done with https://www.businesslist.ph/company/297838/luminaire-printingpublishing-corp
Done with https://www.businesslist.ph/company/305245/peak-outsourcing-philippines
Done with https://www.businesslist.ph/company/299728/draft-groupe
Done with https://www.businesslist.ph/company/300489/infinity-web-solutions
Done with https://www.businesslist.ph/company/264847/skyraiders-travel-services
Done with https://www.businesslist.ph/company/304331/nrk-digital-media
Done with https://www.businesslist.ph/company/299922/pryntcom-copy-center
Done with https://www.businesslist.ph/company/298392/rcla-maru-architects-co
Done with https://www.businesslist.ph/company/138165/mysitemalixi
Done with https://www.businesslist.ph/company/136444/pl-gifts-inc
Done with https://www.businesslist.ph/company/138209/7th-media-design-studio
Done with https://www.businesslist.ph/company/298584/keyframe-media-inc
Done with https://www.businesslist.ph/company/299905/anniego-server
Done with https://www.businesslist.ph/

In [9]:
Travel_Agents = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])
Travel_Agents = extractor('https://www.businesslist.ph/category/travel-agents', 'Travel_Agents', Travel_Agents)
Travel_Agents.to_csv('Travel_Agents.csv')

Done with https://www.businesslist.ph/company/307632/bloomstar-traveltours-novaliches
Done with https://www.businesslist.ph/company/304645/lindela-traveltours
Done with https://www.businesslist.ph/company/264847/skyraiders-travel-services
Done with https://www.businesslist.ph/company/263919/al-jahra-travel
Done with https://www.businesslist.ph/company/302911/travelorder-manila
Done with https://www.businesslist.ph/company/298786/clf-global-online-consultancy-services
Done with https://www.businesslist.ph/company/299821/gazelle-travel-and-tours
Done with https://www.businesslist.ph/company/301709/barilla-travel-and-tours
Done with https://www.businesslist.ph/company/299652/j28-travel-inc
Done with https://www.businesslist.ph/company/265025/skywind-express-travel
Done with https://www.businesslist.ph/company/140303/travbest-traveltours-co
Done with https://www.businesslist.ph/company/297814/lms-tours-and-travel-agency
Done with https://www.businesslist.ph/company/297676/rafflesia-travel-

In [11]:
Plumbing = pd.DataFrame(columns=['Category', 'Name', 'Address', 'Phone', 'Website', 'Rating', 'All Reviews', 'Main Link'])
Plumbing = extractor('https://www.businesslist.ph/category/plumbing-services', 'Plumbing', Plumbing)
Plumbing.to_csv('Plumbing.csv')

Done with https://www.businesslist.ph/company/302967/ad-plumbing-and-electrical-services
Done with https://www.businesslist.ph/company/302133/alphaconstruct-corporation
Done with https://www.businesslist.ph/company/302882/malabanan-siphoning-pozo-negro-bulacan-area
Done with https://www.businesslist.ph/company/299620/malabanan-siphoning-septic-tanktanggal-bara
Done with https://www.businesslist.ph/company/302573/all-manila-malabanan-siphoning-services
Done with https://www.businesslist.ph/company/304031/cebu-septic-tank-corporation
Done with https://www.businesslist.ph/company/303990/nhecys-plumbing-services
Done with https://www.businesslist.ph/company/301976/ras-malabanan-services
Done with https://www.businesslist.ph/company/298372/rta-alarms-and-security-trading-and-services
Done with https://www.businesslist.ph/company/264442/a01rs-plumbing-services
Done with https://www.businesslist.ph/company/303353/gla-malabanan-plumbing-services-736-8402
Done with https://www.businesslist.ph/c

In [ ]:
MAIN_soup.find_all('div', 'company_header')[0].text

In [ ]:
MAIN_soup.find_all('div', 'address')[0].text

In [ ]:
url = f'https://www.businesslist.ph' + MAIN_soup.find_all('div', 'company_header')[0].find('a')['href']

In [ ]:
MAIN_soup.find_all('div', 'company_header')[0].find('a')['href']

In [ ]:
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
soup

In [ ]:
# Extract company name
company_name = soup.find('div', {'id': 'company_name'}).get_text(strip=True)

# Extract address
address = soup.find('div', {'id': 'company_address'}).get_text(strip=True)

# Extract mobile phone
mobile_phone = soup.find_all('div', 'text')[2].text

# Extract email (or the link to send enquiry)
#email_link = soup.find('a', {'title': 'Send Enquiry'})['href']

rating = soup.find_all('span', {'itemprop': 'ratingValue'})[0].text

#Website
f'https://' + soup.find('div', 'text weblinks').text

#All reviews
soup.find_all('div', {'itemprop': 'reviewBody'})[0].text

In [ ]:
mobile_phone

In [ ]:
cont = []
for x in soup.find_all('div', {'itemprop': 'reviewBody'}):
    cont.append(x.text)

In [ ]:
cont